# 🔧 Treinamento de Detector de Ferramentas com YOLO v8

**Notebook otimizado para Google Colab**

Este notebook treina um modelo de detecção de objetos usando YOLOv8 com pré-processamento avançado de imagens.

## 📋 Pré-requisitos:
1. Dataset no formato YOLO (train/valid/test com images/ e labels/)
2. Google Colab com GPU habilitada (**Runtime > Change runtime type > GPU**)
3. Dataset zipado pronto para upload

## 🚀 Passos:
1. Verificar GPU e ambiente
2. Upload do dataset
3. Pré-processamento das imagens
4. Treinamento do modelo
5. Validação e resultados
6. Download do modelo treinado


## 🔍 PASSO 1: Verificar Ambiente e GPU


In [ ]:
# Verificar se está no Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Rodando no Google Colab")
except:
    IN_COLAB = False
    print("⚠️ Não está no Colab")

# Verificar GPU disponível
import torch
print(f"\n🖥️ PyTorch version: {torch.__version__}")
print(f"🎮 CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU não disponível. Vá em: Runtime > Change runtime type > GPU")


## 📦 PASSO 2: Instalar Dependências


In [ ]:
# Instala as bibliotecas necessárias
!pip install -q ultralytics pyyaml opencv-python matplotlib

print("✅ Dependências instaladas!")


In [ ]:
# Importar bibliotecas
import os
import cv2
import shutil
import yaml
import numpy as np
from pathlib import Path
from IPython.display import Image, display
import matplotlib.pyplot as plt

print("✅ Bibliotecas importadas!")


## 💾 PASSO 3: Montar Google Drive (Opcional)

**Se seu dataset estiver no Google Drive, execute esta célula.**


In [ ]:
# Montar Google Drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive montado em /content/drive")
    print("\n📂 Seus arquivos estão em: /content/drive/MyDrive/")


## 📤 PASSO 4: Upload do Dataset

**Execute a célula abaixo para fazer upload do arquivo .zip**


In [ ]:
# Upload do arquivo .zip do dataset
if IN_COLAB:
    from google.colab import files
    print("📤 Faça upload do arquivo .zip do dataset...")
    uploaded = files.upload()
    
    # Pega o nome do arquivo
    dataset_zip = list(uploaded.keys())[0]
    print(f"\n✅ Arquivo recebido: {dataset_zip}")
else:
    # Se não estiver no Colab, defina manualmente
    dataset_zip = "baseScrewdriver.zip"  # ← Altere para o nome do seu arquivo

# Alternativamente, copie do Google Drive (descomente se usar):
# dataset_zip = "/content/drive/MyDrive/seu_dataset.zip"


## 📂 PASSO 5: Descompactar e Verificar Dataset


In [ ]:
# Descompactar o dataset
ORIGINAL_DATA_DIR = "/content/datasets_originais/"

print(f"📦 Descompactando {dataset_zip}...")
!unzip -q {dataset_zip} -d {ORIGINAL_DATA_DIR}

print(f"\n✅ Dataset descompactado em: {ORIGINAL_DATA_DIR}")

# Listar estrutura
print("\n📁 Estrutura do dataset:")
!ls -R {ORIGINAL_DATA_DIR} | head -20


## 🎨 PASSO 6: Pré-Processamento das Imagens

Aplica técnicas avançadas:
- **Redimensionamento** (640x640)
- **CLAHE** (equalização de histograma adaptativa)
- **Gaussian Blur** (redução de ruído)


In [ ]:
def preProcessDataset(base_input_dir, base_output_dir, target_size=(640, 640)):
    """
    Aplica pré-processamento em todo o dataset.
    
    Técnicas aplicadas:
    1. Redimensionamento para tamanho padrão
    2. CLAHE (Contrast Limited Adaptive Histogram Equalization)
    3. Gaussian Blur para redução de ruído
    """
    print("\n🎨 Iniciando pré-processamento do dataset...\n")
    
    total_processed = 0
    
    # Itera sobre train, valid, test
    for split in ['train', 'valid', 'test']:
        input_images_path = os.path.join(base_input_dir, split, 'images')
        input_labels_path = os.path.join(base_input_dir, split, 'labels')
        
        output_images_path = os.path.join(base_output_dir, split, 'images')
        output_labels_path = os.path.join(base_output_dir, split, 'labels')
        
        # Criar pastas de saída
        os.makedirs(output_images_path, exist_ok=True)
        os.makedirs(output_labels_path, exist_ok=True)
        
        # Processar imagens
        if os.path.exists(input_images_path):
            images = [f for f in os.listdir(input_images_path) 
                     if f.endswith(('.jpg', '.jpeg', '.png'))]
            
            print(f"📂 Processando {split.upper()}: {len(images)} imagens")
            
            for idx, filename in enumerate(images):
                img_path = os.path.join(input_images_path, filename)
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"   ⚠️ Erro ao ler: {filename}")
                    continue
                
                # ETAPA 1: Redimensionamento
                processed_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
                
                # ETAPA 2: CLAHE (Realce de Contraste)
                lab = cv2.cvtColor(processed_image, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                l_clahe = clahe.apply(l)
                lab_clahe = cv2.merge((l_clahe, a, b))
                processed_image = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)
                
                # ETAPA 3: Redução de Ruído
                processed_image = cv2.GaussianBlur(processed_image, (5, 5), 0)
                
                # Salvar imagem processada
                output_path = os.path.join(output_images_path, filename)
                cv2.imwrite(output_path, processed_image)
                
                total_processed += 1
                
                # Mostrar progresso a cada 50 imagens
                if (idx + 1) % 50 == 0:
                    print(f"   📊 Progresso: {idx + 1}/{len(images)}")
        
        # Copiar labels (não precisam ser modificados)
        if os.path.exists(input_labels_path):
            labels = os.listdir(input_labels_path)
            for label_file in labels:
                shutil.copy(
                    os.path.join(input_labels_path, label_file),
                    output_labels_path
                )
            print(f"   ✅ {len(labels)} labels copiados\n")
    
    print(f"\n✅ Pré-processamento concluído!")
    print(f"📊 Total de imagens processadas: {total_processed}")
    
    return total_processed

# Executar pré-processamento
PROCESSED_DATA_DIR = "/content/datasets_processados/"
total = preProcessDataset(ORIGINAL_DATA_DIR, PROCESSED_DATA_DIR)


## 📝 PASSO 7: Configurar data.yaml


In [ ]:
# Copiar e ajustar o data.yaml
original_yaml_path = os.path.join(ORIGINAL_DATA_DIR, 'data.yaml')
processed_yaml_path = os.path.join(PROCESSED_DATA_DIR, 'data.yaml')

print("📝 Configurando data.yaml...")

with open(original_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

# Atualizar caminhos para dados processados
data_config['path'] = PROCESSED_DATA_DIR
data_config['train'] = 'train/images'
data_config['val'] = 'valid/images'
data_config['test'] = 'test/images'

# Salvar novo yaml
with open(processed_yaml_path, 'w') as f:
    yaml.dump(data_config, f)

print(f"\n✅ Arquivo data.yaml criado!")
print(f"\n📋 Classes: {data_config.get('names', [])}")
print(f"🔢 Total: {data_config.get('nc', 0)} classes")


## 🚀 PASSO 8: Treinamento do Modelo

**IMPORTANTE:** Este processo pode levar de **30 minutos a 2 horas** dependendo:
- Tamanho do dataset
- Número de épocas
- GPU disponível


In [ ]:
from ultralytics import YOLO

print("🚀 Iniciando treinamento do modelo YOLO...\n")
print("⏱️ Isso pode levar algum tempo. Aguarde...\n")

# Carregar modelo pré-treinado
model = YOLO('yolov8n.pt')  # Nano (mais rápido)
# model = YOLO('yolov8s.pt')  # Small (mais preciso)
# model = YOLO('yolov8m.pt')  # Medium (ainda mais preciso)

print("✅ Modelo YOLOv8 Nano carregado\n")

# Configuração do treinamento
results = model.train(
    data=processed_yaml_path,      # Caminho para data.yaml
    epochs=50,                      # Número de épocas
    imgsz=640,                      # Tamanho da imagem
    batch=16,                       # Tamanho do batch
    project='meus_treinamentos',    # Nome do projeto
    name='detector_ferramentas_v1', # Nome do experimento
    patience=10,                    # Early stopping
    save=True,                      # Salvar checkpoints
    device=0 if torch.cuda.is_available() else 'cpu',  # GPU ou CPU
    workers=2,                      # Número de workers
    verbose=True                    # Mostrar logs detalhados
)

print("\n🎉 Treinamento concluído!")


## 📊 PASSO 9: Visualizar Resultados


In [ ]:
# Caminho dos resultados
results_dir = "/content/meus_treinamentos/detector_ferramentas_v1"

print("📊 Resultados do Treinamento:\n")

# Mostrar gráficos de treinamento
plots = [
    'results.png',           # Métricas gerais
    'confusion_matrix.png',  # Matriz de confusão
    'val_batch0_pred.jpg',   # Predições no validation
]

for plot in plots:
    plot_path = os.path.join(results_dir, plot)
    if os.path.exists(plot_path):
        print(f"\n📈 {plot}:")
        display(Image(filename=plot_path))
    else:
        print(f"⚠️ {plot} não encontrado")


## 🧪 PASSO 10: Validar Modelo


In [ ]:
# Carregar o melhor modelo treinado
best_model_path = os.path.join(results_dir, 'weights', 'best.pt')

if os.path.exists(best_model_path):
    print(f"✅ Modelo encontrado: {best_model_path}\n")
    
    # Carregar modelo
    model_trained = YOLO(best_model_path)
    
    # Validar no dataset de teste
    print("🧪 Validando modelo no dataset de teste...\n")
    metrics = model_trained.val(data=processed_yaml_path, split='test')
    
    print("\n📊 Métricas de Validação:")
    print(f"   mAP50: {metrics.box.map50:.4f}")
    print(f"   mAP50-95: {metrics.box.map:.4f}")
    print(f"   Precision: {metrics.box.mp:.4f}")
    print(f"   Recall: {metrics.box.mr:.4f}")
else:
    print("❌ Modelo não encontrado!")


## 🎯 PASSO 11: Testar em Imagens de Exemplo


In [ ]:
# Testar em imagens de exemplo
test_images_dir = os.path.join(PROCESSED_DATA_DIR, 'test', 'images')

if os.path.exists(test_images_dir):
    test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))][:3]
    
    if test_images:
        print("🎯 Testando modelo em imagens de exemplo...\n")
        
        for img_name in test_images:
            img_path = os.path.join(test_images_dir, img_name)
            
            # Fazer predição
            results = model_trained.predict(
                source=img_path,
                conf=0.5,  # Confiança mínima 50%
                save=True,
                project='predicoes_teste',
                name='resultados'
            )
            
            print(f"\n✅ {img_name}:")
            for r in results:
                print(f"   Detectados: {len(r.boxes)} objetos")
                for box in r.boxes:
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    class_name = model_trained.names[cls]
                    print(f"   - {class_name}: {conf:.2%}")
        
        # Mostrar resultados
        print("\n🖼️ Imagens com detecções:")
        pred_dir = "/content/predicoes_teste/resultados"
        if os.path.exists(pred_dir):
            for img in os.listdir(pred_dir):
                if img.endswith(('.jpg', '.jpeg', '.png')):
                    display(Image(filename=os.path.join(pred_dir, img)))
else:
    print("⚠️ Nenhuma imagem de teste encontrada")


## 💾 PASSO 12: Download do Modelo Treinado


In [ ]:
# Download do modelo treinado
if IN_COLAB and os.path.exists(best_model_path):
    from google.colab import files
    
    print("💾 Preparando download do modelo...\n")
    
    # Copiar para um nome mais amigável
    model_download_path = "/content/modelo_detector_ferramentas.pt"
    shutil.copy(best_model_path, model_download_path)
    
    print("📥 Fazendo download do modelo...")
    files.download(model_download_path)
    
    print("\n✅ Download concluído!")
    print(f"📊 Tamanho: {os.path.getsize(model_download_path) / (1024*1024):.2f} MB")
else:
    print("ℹ️ Para download manual, localize:")
    print(f"   {best_model_path}")


## 💾 PASSO 13: Salvar no Google Drive (Opcional)


In [ ]:
# Salvar modelo no Google Drive
if IN_COLAB:
    save_to_drive = input("💾 Salvar modelo no Google Drive? (s/n): ")
    
    if save_to_drive.lower() == 's':
        # Criar pasta no Drive
        drive_save_path = "/content/drive/MyDrive/Modelos_YOLO/"
        os.makedirs(drive_save_path, exist_ok=True)
        
        # Copiar modelo
        final_path = os.path.join(drive_save_path, "detector_ferramentas_best.pt")
        shutil.copy(best_model_path, final_path)
        
        print(f"\n✅ Modelo salvo no Drive: {final_path}")
        
        # Também salvar resultados
        results_zip = "/content/resultados_treinamento.zip"
        !zip -r -q {results_zip} {results_dir}
        shutil.copy(results_zip, drive_save_path)
        print(f"✅ Resultados salvos: {drive_save_path}resultados_treinamento.zip")


## 📋 RESUMO FINAL


In [ ]:
print("\n" + "="*70)
print(" "*20 + "📊 RESUMO DO TREINAMENTO")
print("="*70)

print(f"\n✅ Dataset:")
print(f"   - Imagens processadas: {total} imagens")
print(f"   - Classes: {data_config.get('nc', 0)}")

print(f"\n✅ Modelo:")
print(f"   - Arquitetura: YOLOv8 Nano")
print(f"   - Épocas: 50")
print(f"   - Caminho: {best_model_path}")

print(f"\n✅ Próximos Passos:")
print(f"   1. Baixe o modelo best.pt")
print(f"   2. Use no seu projeto Flask/Python")
print(f"   3. Substitua o modelo no Roboflow")

print("\n" + "="*70)
print("\n🎉 Treinamento concluído com sucesso!")
print("="*70 + "\n")
